<a href="https://colab.research.google.com/github/billbch/TP1/blob/main/Inference_Speed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Export YOLOv8

In [ ]:
!pip install ultralytics

In [ ]:
!pip install torch==2.0.1+cu118

In [ ]:
#from ultralytics import YOLO
import cv2
import torch
from PIL import Image

In [ ]:
!pip freeze > requirements.txt

In [ ]:
print(torch.__version__)

2.0.1+cu118


In [ ]:
model = YOLO('yolov8_sgrieta.pt')  # load a custom trained
# Export the model
model.export(format='tflite')
print(model.model.names)


In [ ]:
model = YOLO('yolov8_grieta.pt')  # load a custom trained
# Export the model
#model.export(format='tflite')
print(model.model.names)
results = model.predict("/content/images/IMG_7395.JPG", save=True)
print(results)

In [ ]:
from pathlib import Path

class Colors:
    # Ultralytics color palette https://ultralytics.com/
    def __init__(self):
        # hex = matplotlib.colors.TABLEAU_COLORS.values()
        hex = ('FF3838', 'FF9D97', 'FF701F', 'FFB21D', 'CFD231', '48F90A', '92CC17', '3DDB86', '1A9334', '00D4BB',
               '2C99A8', '00C2FF', '344593', '6473FF', '0018EC', '8438FF', '520085', 'CB38FF', 'FF95C8', 'FF37C7')
        self.palette = [self.hex2rgb('#' + c) for c in hex]
        self.n = len(self.palette)

    def __call__(self, i, bgr=False):
        c = self.palette[int(i) % self.n]
        return (c[2], c[1], c[0]) if bgr else c

    @staticmethod
    def hex2rgb(h):  # rgb order (PIL)
        return tuple(int(h[1 + i:1 + i + 2], 16) for i in (0, 2, 4))


colors = Colors()

def plot_one_box(x, im, color=(128, 128, 128), label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def display(imgs, pred, names, files, pprint=False, show=False, save=False, crop=False, render=False, save_dir=Path('')):
    lbl_count = {}
    for i, (im, pred) in enumerate(zip(imgs, pred)):
        str = f'image {i + 1}/{len(pred)}: {im.shape[0]}x{im.shape[1]} '
        if pred is not None:
            for c in pred[:, -1].unique():
                n = (pred[:, -1] == c).sum()  # detections per class
                str += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
                lbl_count[names[int(c)]] = n.item()
            if show or save or render or crop:
                for *box, conf, cls in pred:  # xyxy, confidence, class
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(box, im, label=label, color=colors(cls))

        im = Image.fromarray(im.astype(np.uint8)) if isinstance(im, np.ndarray) else im  # from np
        print(str.rstrip(', '))
        f = files[i]
        im.save(save_dir / f)  # save
        opencvImage = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
        _, im_arr = cv2.imencode('.jpg', opencvImage)  # im_arr: image in Numpy one-dim array format.
        im_bytes = im_arr.tobytes()
        im_b64 = base64.b64encode(im_bytes)
        #print(f"{'Saved' * (i == 0)} {f}", end=',' if i < self.n - 1 else f' to {save_dir}\n')
    for i in names.values():
        if i not in lbl_count.keys():
            lbl_count[i] = 0
    return lbl_count, im_b64

In [ ]:
!pip install -U ultralytics

In [ ]:
import cv2
import torch
from PIL import Image
import ultralytics

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5_grieta.pt')  # local model
im1 = cv2.imread("/content/images/IMG_7208.JPG")[..., ::-1]
results = model(im1, size=640) # batch of images

#print(results.f)
#print(results.ims)
lbl, b64 = display(results.ims, results.pred, results.names, results.files, save=True)
# Results
#results.print()
#results.save(save_dir="./im/", exist_ok=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-15 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 


image 1/2: 960x720 2 Grietas


In [ ]:
lbl

{'Grieta': 2, 'Hueco': 0, 'HuecoGrave': 0}

In [ ]:
b64

b'...'

In [ ]:
print(results.names)
print(results.files)
#im = Image.fromarray(results.pred[0].astype(np.uint8))

{0: 'Hueco', 1: 'Grieta', 2: 'HuecoGrave'}
['image0.jpg']


# Export YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt

In [ ]:
%cd yolov5

In [ ]:
!python export.py --weights yolov5_sgrieta.pt --include tfjs saved_model

In [ ]:
!python export.py --weights yolov5_grieta.pt --include tflite

In [ ]:
%cd best_web_model/
%cat group1-shard1of7.bin group1-shard2of7.bin group1-shard3of7.bin group1-shard4of7.bin group1-shard5of7.bin group1-shard6of7.bin group1-shard7of7.bin > weights.bin

/content/yolov5/best_web_model


In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5_sgrieta',force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
print(model.model.names)
print(model.names)


model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5_grieta',force_reload=True, trust_repo=True)  # or yolov5n - yolov5x6 or custom
print(model.model.names)
print(model.names)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


{0: 'Hueco', 1: 'HuecoGrave'}
{0: 'Hueco', 1: 'HuecoGrave'}


YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


{0: 'Hueco', 1: 'Grieta', 2: 'HuecoGrave'}
{0: 'Hueco', 1: 'Grieta', 2: 'HuecoGrave'}


#TFLITE to tfjs

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model ./model/ ./model/

2023-09-05 18:08:07.730395: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 18:08:08.929871: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-05 18:08:16.976287: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-09-05 18:08:16.976461: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session


In [ ]:
import shutil
shutil.make_archive("tfjs_model", 'zip', "output_directory")

# Inference

In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
print(cv2.__version__)
print(np.__version__)

4.8.0
1.23.5


In [ ]:
import os
import cv2
import numpy as np
import sys
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter
import json
import matplotlib
import matplotlib.pyplot as plt
import time
import base64

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, image_path, lblpath, min_conf=0.5, savepath='/content/results', txt_only=False):
  start_time = time.time()

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]
  #print(width)
  #print(height)

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Load image and resize to expected shape [1xHxWx3]
  image = cv2.imread(image_path)
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  imH, imW, _ = image.shape
  image_resized = cv2.resize(image_rgb, (width, height))
  input_data = np.expand_dims(image_resized, axis=0)

  # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
  if float_input:
    input_data = (np.float32(input_data) - input_mean) / input_std

  # Perform the actual detection by running the model with the image as input
  interpreter.set_tensor(input_details[0]['index'],input_data)
  interpreter.invoke()

  # Retrieve detection results
  #print(interpreter.get_tensor(output_details[0]['index']))
  boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
  classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
  scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

  for i in range(len(scores)):
    if ((scores[i] > min_conf) and (scores[i] <= 1.0)):
        # Get bounding box coordinates and draw box
        # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
        ymin = int(max(1,(boxes[i][0] * imH)))
        xmin = int(max(1,(boxes[i][1] * imW)))
        ymax = int(min(imH,(boxes[i][2] * imH)))
        xmax = int(min(imW,(boxes[i][3] * imW)))

        cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

        # Draw label
        object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
        label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
        labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
        label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
        cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
        cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

  #image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  _, im_arr = cv2.imencode('.jpg', image)  # im_arr: image in Numpy one-dim array format.
  im_bytes = im_arr.tobytes()
  im_b64 = base64.b64encode(im_bytes)

  end_time = time.time()
  # Calculate elapsed time
  elapsed_time = end_time- start_time
  print("Elapsed time: ", elapsed_time*1000)

  return elapsed_time*1000
  #print(im_b64)
  #print(type(im_b64))
  #print(str(im_b64))
  #print(type(str(im_b64)))
  #im_bytes = base64.b64decode(im_b64)
  #im_arr = np.frombuffer(im_bytes, dtype=np.uint8)  # im_arr is one-dim Numpy array
  #img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
  #plt.figure(figsize=(12,16))
  #plt.imshow(img)
  #plt.show()

In [ ]:
# Set up variables for running user's model
import glob
import random

imgpath = "/content/images"
images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')
#print(images)
labels = ['labelmap_grietas.txt', 'labelmap_sgrietas.txt']
models = ['detect_grietas.tflite', 'detect_sgrietas.tflite']

times = []

for i in range(2):
    tim = []
    for im in images[0:50]:
        PATH_TO_IMAGES= im   # Path to test images folder
        PATH_TO_MODEL= '/content/' + models[i]   # Path to .tflite model file
        PATH_TO_LABELS='/content/' + labels[i]   # Path to labelmap.txt file
        min_conf_threshold=0.15   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)

        # Run inferencing function!
        t = tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold)
        tim.append(t)
    times.append(tim)

Elapsed time:  124.20392036437988
Elapsed time:  102.63538360595703
Elapsed time:  86.2884521484375
Elapsed time:  89.22171592712402
Elapsed time:  90.3627872467041
Elapsed time:  91.21036529541016
Elapsed time:  87.47148513793945
Elapsed time:  88.24777603149414
Elapsed time:  90.29150009155273
Elapsed time:  89.23196792602539
Elapsed time:  91.86124801635742
Elapsed time:  85.0989818572998
Elapsed time:  86.79699897766113
Elapsed time:  87.23688125610352
Elapsed time:  86.3957405090332
Elapsed time:  86.50994300842285
Elapsed time:  87.39924430847168
Elapsed time:  90.99578857421875
Elapsed time:  84.20848846435547
Elapsed time:  87.19873428344727
Elapsed time:  88.86003494262695
Elapsed time:  87.8136157989502
Elapsed time:  80.22236824035645
Elapsed time:  87.68796920776367
Elapsed time:  86.91573143005371
Elapsed time:  89.42341804504395
Elapsed time:  85.90078353881836
Elapsed time:  94.88391876220703
Elapsed time:  86.99870109558105
Elapsed time:  87.26000785827637
Elapsed time:

In [ ]:
print("Min", min(times[0]))
print("15", np.percentile(times[0], 15))
print("AVG", sum(times[0])/len(times[0]))
print("85", np.percentile(times[0], 85))
print("MAX", max(times[0]))

print()

print("Min", min(times[1]))
print("15", np.percentile(times[1], 15))
print("AVG", sum(times[1])/len(times[1]))
print("85", np.percentile(times[1], 85))
print("MAX", max(times[1]))

Min 75.4704475402832
15 84.91743803024292
AVG 87.93317794799805
85 90.49838781356812
MAX 124.20392036437988

Min 72.66902923583984
15 84.17294025421143
AVG 86.64217948913574
85 89.84949588775635
MAX 94.7122573852539


In [ ]:
# Set up variables for running user's model
PATH_TO_IMAGES='/content/images/IMG_7236.JPG'   # Path to test images folder
PATH_TO_MODEL='/content/model/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/model/labelmap.txt'   # Path to labelmap.txt file
min_conf_threshold=0.15   # Confidence threshold (try changing this to 0.01 if you don't see any detection results)

# Run inferencing function!
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold)

In [ ]:
 %%timeit
 tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold)

In [ ]:
from tensorflow.lite.python.interpreter import Interpreter

interpreter = Interpreter(model_path='/content/detect.tflite')
print(interpreter.allocate_tensors())

input_details = interpreter.get_input_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
print(width)
print(height)

None
300
300


# INFERENCE EXPERIMENT

## YOLO v8

In [ ]:
print(results[0].speed)
print(results[0].speed['inference'])

{'preprocess': 3.4933090209960938, 'inference': 254.49371337890625, 'postprocess': 1.220703125}
254.49371337890625


In [ ]:
import glob
import numpy as np

imgpath = "/content/images"
images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

In [ ]:
print("YOLOV8 - Grietas")
times8 = []
for i in images[0:50]:
    model = YOLO('yolov8_grieta.pt')  # load a custom trained
    results = model.predict(i) # "/content/images/IMG_7395.JPG" , save=True
    #print(results[0].speed['inference'])
    times8.append(results[0].speed['inference'])

print("--------------------------------------------------------------------------------------------------------")
print("------------------------------Mean Inference Time for Yolov8 - Grietas:", sum(times8)/len(times8))
print("--------------------------------------------------------------------------------------------------------")
print()
print("YOLOV8 - Sin Grietas")
times8s = []
for i in images[0:50]:
    model = YOLO('yolov8_sgrieta.pt')  # load a custom trained
    results = model.predict(i)
    #print(results[0].speed['inference'])
    times8s.append(results[0].speed['inference'])
print("--------------------------------------------------------------------------------------------------------")
print("---------------------------------Mean Inference Time for Yolov8 - Sin Grietas:", sum(times8s)/len(times8s))
print("--------------------------------------------------------------------------------------------------------")
#print(results)

YOLOV8 - Grietas



image 1/1 /content/images/IMG_7282_3.jpg: 736x576 6 Huecos, 2 Grietas, 233.3ms
Speed: 3.4ms preprocess, 233.3ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7319_3.jpg: 736x576 2 Grietas, 1 HuecoGrave, 186.5ms
Speed: 3.7ms preprocess, 186.5ms inference, 1.0ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7244_3.jpg: 736x576 1 Hueco, 1 Grieta, 3 HuecoGraves, 191.5ms
Speed: 3.5ms preprocess, 191.5ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7338_2.jpg: 736x576 2 Grietas, 146.6ms
Speed: 2.4ms preprocess, 146.6ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7242_4.jpg: 736x576 1 Hueco, 4 HuecoGraves, 139.7ms
Speed: 2.3ms preprocess, 139.7ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7388_3.jpg: 736x576 8 Huecos, 2 Grietas, 138.5ms
Speed: 2.3ms preproc

--------------------------------------------------------------------------------------------------------
------------------------------Mean Inference Time for Yolov8 - Grietas: 158.89910697937012
--------------------------------------------------------------------------------------------------------

YOLOV8 - Sin Grietas



image 1/1 /content/images/IMG_7282_3.jpg: 736x576 6 Huecos, 144.5ms
Speed: 2.5ms preprocess, 144.5ms inference, 0.8ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7319_3.jpg: 736x576 1 HuecoGrave, 144.7ms
Speed: 2.4ms preprocess, 144.7ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7244_3.jpg: 736x576 1 Hueco, 3 HuecoGraves, 175.4ms
Speed: 3.2ms preprocess, 175.4ms inference, 0.9ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7338_2.jpg: 736x576 (no detections), 191.8ms
Speed: 3.6ms preprocess, 191.8ms inference, 0.7ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7242_4.jpg: 736x576 1 Hueco, 4 HuecoGraves, 185.1ms
Speed: 3.3ms preprocess, 185.1ms inference, 1.1ms postprocess per image at shape (1, 3, 736, 576)

image 1/1 /content/images/IMG_7388_3.jpg: 736x576 7 Huecos, 136.6ms
Speed: 2.3ms preprocess, 136.6ms inference, 0.8ms postpro

--------------------------------------------------------------------------------------------------------
---------------------------------Mean Inference Time for Yolov8 - Sin Grietas: 156.1700677871704
--------------------------------------------------------------------------------------------------------


In [ ]:
print("Min", min(times8))
print("15", np.percentile(times8, 15))
print("AVG", sum(times8)/len(times8))
print("85", np.percentile(times8, 85))
print("MAX", max(times8))

print()

print("Min", min(times8s))
print("15", np.percentile(times8s, 15))
print("AVG", sum(times8s)/len(times8s))
print("85", np.percentile(times8s, 85))
print("MAX", max(times8s))

Min 131.77824020385742
15 137.63068914413452
AVG 158.89910697937012
85 187.58561611175537
MAX 261.584997177124

Min 132.9488754272461
15 137.09022998809814
AVG 156.1700677871704
85 183.7794065475464
MAX 228.12271118164062


## YOLO v5

In [ ]:
results.t

(8.099079132080078, 191.0378932952881, 0.7047653198242188)

In [ ]:
import cv2
import torch
from PIL import Image

print("YOLOV5 - Grietas")
times5 = []
for i in images[0:50]:
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5_grieta.pt')  # local model
    im1 = cv2.imread(i)[..., ::-1]
    results = model(i) # batch of images
    times5.append(results.t[1])
print("--------------------------------------------------------------------------------------------------------")
print("------------------------Mean Inference Time for Yolov5 - Grietas:", sum(times5)/len(times5))
print("--------------------------------------------------------------------------------------------------------")
print()

print("YOLOV5 - Grietas")
times5s = []
for i in images[0:50]:
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5_sgrieta.pt')  # local model
    im1 = cv2.imread(i)[..., ::-1]
    results = model(i) # batch of images
    times5s.append(results.t[1])
print("--------------------------------------------------------------------------------------------------------")
print("---------------------------Mean Inference Time for Yolov5 - Sin Grietas:", sum(times5s)/len(times5s))
print("--------------------------------------------------------------------------------------------------------")

print(results.t)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 


YOLOV5 - Grietas


Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients
Adding AutoShape... 
Using cache found in

--------------------------------------------------------------------------------------------------------
------------------------Mean Inference Time for Yolov5 - Grietas: 208.98910999298096
--------------------------------------------------------------------------------------------------------

YOLOV5 - Grietas


Model summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-9-14 Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 
Using cache found in

--------------------------------------------------------------------------------------------------------
---------------------------Mean Inference Time for Yolov5 - Sin Grietas: 207.85858631134033
--------------------------------------------------------------------------------------------------------
(7.622241973876953, 185.47844886779785, 0.6499290466308594)


In [ ]:
results

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 960x720 5 Huecos
Speed: 8.0ms pre-process, 189.7ms inference, 0.7ms NMS per image at shape (1, 3, 640, 480)

In [ ]:
print("Min", min(times5))
print("15", np.percentile(times5, 15))
print("AVG", sum(times5)/len(times5))
print("85", np.percentile(times5, 85))
print("MAX", max(times5))

print()

print("Min", min(times5s))
print("15", np.percentile(times5s, 15))
print("AVG", sum(times5s)/len(times5s))
print("85", np.percentile(times5s, 85))
print("MAX", max(times5s))

Min 180.99308013916016
15 184.42003726959229
AVG 208.98910999298096
85 250.0234603881836
MAX 258.8536739349365

Min 178.786039352417
15 184.68352556228638
AVG 207.85858631134033
85 248.8381028175354
MAX 254.63390350341797


In [ ]:
print(times[0])
print(times[1])
print(times8)
print(times8s)
print(times5)
print(times5s)

[124.20392036437988, 102.63538360595703, 86.2884521484375, 89.22171592712402, 90.3627872467041, 91.21036529541016, 87.47148513793945, 88.24777603149414, 90.29150009155273, 89.23196792602539, 91.86124801635742, 85.0989818572998, 86.79699897766113, 87.23688125610352, 86.3957405090332, 86.50994300842285, 87.39924430847168, 90.99578857421875, 84.20848846435547, 87.19873428344727, 88.86003494262695, 87.8136157989502, 80.22236824035645, 87.68796920776367, 86.91573143005371, 89.42341804504395, 85.90078353881836, 94.88391876220703, 86.99870109558105, 87.26000785827637, 75.4704475402832, 84.70678329467773, 85.43705940246582, 86.20691299438477, 88.70530128479004, 83.99128913879395, 86.17568016052246, 86.27867698669434, 88.5627269744873, 90.69633483886719, 86.5633487701416, 85.23702621459961, 85.75272560119629, 90.57140350341797, 85.90054512023926, 86.77124977111816, 78.00912857055664, 84.8245620727539, 85.08992195129395, 82.87382125854492]
[94.38109397888184, 88.26422691345215, 91.53223037719727